autograd 包为张量上的所有操作提供了自动求导机制。它是一个在运行时定义(define-by-run）的框架，这意味着反向传播是根据代码如何运行来决定的，并且每次迭代可以是不同的.

# 张量

torch.Tensor 是这个包的核心类。如果设置它的属性 .requires_grad 为 True，那么它将会追踪对于该张量的所有操作。当完成计算后可以通过调用 .backward()，来自动计算所有的梯度。这个张量的所有梯度将会自动累加到.grad属性.

要阻止一个张量被跟踪历史，可以调用 .detach() 方法将其与计算历史分离，并阻止它未来的计算记录被跟踪。

为了防止跟踪历史记录(和使用内存），可以将代码块包装在 with torch.no_grad(): 中。在评估模型时特别有用，因为模型可能具有 requires_grad = True 的可训练的参数，但是我们不需要在此过程中对他们进行梯度计算。

还有一个类对于autograd的实现非常重要：Function。

Tensor 和 Function 互相连接生成了一个无圈图(acyclic graph)，它编码了完整的计算历史。每个张量都有一个 .grad_fn 属性，该属性引用了创建 Tensor 自身的Function(除非这个张量是用户手动创建的，即这个张量的 grad_fn 是 None )。

如果需要计算导数，可以在 Tensor 上调用 .backward()。如果 Tensor 是一个标量(即它包含一个元素的数据），则不需要为 backward() 指定任何参数，但是如果它有更多的元素，则需要指定一个 gradient 参数，该参数是形状匹配的张量。

In [1]:
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [2]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [3]:
# y 作为操作的结果被创建，所以它有 grad_fn.
print(y.grad_fn)

In [4]:
# 在 y 上进行更多操作
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


 .requires_grad_( ... ) 可以改变现有张量的 requires_grad 属性。如果没有指定的话，默认输入的这个标志是 False。

In [5]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


# 梯度

现在开始进行反向传播，因为 out 是一个标量，因此 out.backward() 和 out.backward(torch.tensor(1.)) 等价。

In [7]:
out.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


结果应该就是得到数值都是 4.5 的矩阵。这里我们用 o 表示 out 变量，那么根据之前的定义会有：
$$\left. \begin{array}  { l  }  { 0 = \frac { 1 } { 4 } \sum _ { i = } ^ { z _ { i } } z _ { i } } \\ { z _ { i } = 3 ( x _ { i } + 2 ) ^ { 2 } } \\ { z _ { i } | _ { i } | x _ { i } = 27 } \end{array} \right.$$

详细来说，初始定义的 x 是一个全为 1 的矩阵，然后加法操作 x+2 得到 y ，接着 y*y*3， 得到 z ，并且此时 z 是一个 2*2 的矩阵，所以整体求平均得到 out 变量应该是除以 4，所以得到上述三条公式。
因此，计算梯度：
$$\left. \begin{array}  { l  }  { \frac { \partial 0 } { \partial x _ { 1 } } = \frac { 3 } { 2 } ( x _ { i } + 2 ) } \\ { \frac { \partial _ { i } } { \partial x _ { i } } | _ { x _ { i } } = \frac { 9 } { 2 } = 4.5 } \end{array} \right.$$

从数学上来说，如果你有一个向量值函数：
$$\overrightarrow { y } = f ( \overrightarrow { x } ) $$

那么对应的梯度是一个雅克比矩阵(Jacobian matrix)：
一般来说，torch.autograd 就是用于计算雅克比向量(vector-Jacobian)乘积的工具。这里略过数学公式，直接上代码例子介绍：

In [9]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000: # norm() 范数
    y = y * 2
print(y)

tensor([ 144.7906, -929.8047, 1286.4860], grad_fn=<MulBackward0>)


现在在这种情况下，y 不再是一个标量。torch.autograd 不能够直接计算完整的雅克比矩阵，但是如果我们只想要雅克比向量积，只需要简单的传递向量给 backward 作为参数。

In [10]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)
print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


也可以通过将代码块包装在 with torch.no_grad()，来阻止autograd跟踪设置了 .requires_grad=True 的张量的历史记录。

In [11]:
print(x.requires_grad)
print((x ** 2).requires_grad)
with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False
